In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib
from talib import ATR
import imblearn
from imblearn.over_sampling import SMOTE

In [2]:
def MA(df):
    df['signal'] = 0.0
    df['ma5'] = df['close'].rolling(window=390, min_periods=1, center=False).mean()
    df['ma10'] = df['close'].rolling(window=780, min_periods=1, center=False).mean()
    df['signal'][780:] = np.where(df['ma5'][780:] > df['ma10'][780:], 1.0, 0.0)
    df['MA_signal'] = df['signal'].diff()
    return df['MA_signal']
def MA_price(df): 
    df['MA_signal_price']=df['MA_signal']*df['close']
    return df['MA_signal_price']
    
def MA_plot(df):
    # Initialize the plot figure
    fig = plt.figure(figsize=(20,10))

    # Add a subplot and label for y-axis
    ax1 = fig.add_subplot(111,  ylabel='Price in $')

    # Plot the closing price
    df['close'].plot(ax=ax1, lw=1.)

    # Plot the short and long moving averages
    df.plot(x='datetime',y=['ma5', 'ma10'],ax=ax1, lw=2.)

    # Plot the buy signals
    ax1.plot(df.loc[df.MA_signal == 1.0].index, 
             df.ma5[df.MA_signal == 1.0],
             '^', markersize=15, color='green')

    # Plot the sell signals
    ax1.plot(df.loc[df.MA_signal == -1.0].index, 
             df.ma5[df.MA_signal == -1.0],
             'v', markersize=15, color='red')

    # Show the plot
    plt.show()

In [3]:
def MACD(df):
    df['SMA'] = talib.SMA(df['close'], timeperiod = 3510)
    df['EMA12'] = talib.EMA(df['close'], timeperiod = 4680)
    df['EMA26'] = talib.EMA(df['close'], timeperiod = 10140)
    df['DIF'] = df['EMA12'] - (df['EMA26'])
    df['DEA'] = talib.EMA(df['DIF'], timeperiod = 9)
    df['MACD']=2*(df['DIF']-df['DEA'])
    df['MACD_signal']=0
    df.drop(['MACD_signal'],axis=1,inplace=True)
    DIF = df['DIF']
    DEA = df['DEA']
    
    MACD_signal = np.zeros(df.shape[0])

    for i in range(len(DIF)-1):

        if(df['DIF'][i] > 0 and df['DEA'][i] > 0 and df['DIF'][i+1] > df['DIF'][i] and df['DEA'][i+1] > df['DEA'][i]):
            MACD_signal[i] = 1.0


        elif(df['DIF'][i] < 0 and df['DEA'][i] < 0 and df['DIF'][i+1] > df['DIF'][i] and df['DEA'][i+1] > df['DEA'][i]):
            MACD_signal[i] = 1.0

        elif(df['DIF'][i] < 0 and df['DEA'][i] < 0 and df['DIF'][i+1] < df['DIF'][i] and df['DEA'][i+1] < df['DEA'][i]):
            MACD_signal[i] = -1.0

        elif(df['DIF'][i] > 0 and df['DEA'][i] > 0 and df['DIF'][i+1] < df['DIF'][i] and df['DEA'][i+1] < df['DEA'][i]):
            MACD_signal[i] = -1.0

        else:
            MACD_signal[i] = 0.0
    
    for i in range(1,len(DIF)): 
        if(MACD_signal[i-1] != 1.0 and MACD_signal[i] == 1.0):
            MACD_signal[i-1] = 1.0
        elif(MACD_signal[i-1] != -1.0 and MACD_signal[i] == -1.0):
            MACD_signal[i-1] = -1.0
        else:
            MACD_signal[i-1] = 0.0

    df.insert(1,'MACD_signal', MACD_signal)

    return df['MACD_signal']
def MACD_price(df): 
    df['MACD_signal_price']=df['MACD_signal']*df['close']
    return df['MACD_signal_price']


def MACD_plot(df):
    # Initialize the plot figure
    fig = plt.figure(figsize=(20,10))

    # Add a subplot and label for y-axis
    ax1 = fig.add_subplot(111,  ylabel='Price in $')

    # Plot the closing price
    df.plot(x='datetime',y='close',ax=ax1, lw=1.)

    # Plot the short and long moving averages
    #df.plot(x='Time',y=['ma9', 'ma18'],ax=ax1, lw=2.)

    # Plot the buy signals
    ax1.plot(df.loc[df.MACD_signal == 1.0].index, 
             df.close[df.MACD_signal == 1.0],
             '^', markersize=15, color='green')

    # Plot the sell signals
    ax1.plot(df.loc[df.MACD_signal == -1.0].index, 
             df.close[df.MACD_signal == -1.0],
             'v', markersize=15, color='red')

    # Show the plot
    plt.show()


In [4]:
def BBAND(df):
    df['BBand_upper_band'], df['BBand_middle_band'], df['BBand_lower_band'] = talib.BBANDS(df['close'], timeperiod =3900,nbdevup=2, nbdevdn=2)
    df['PercentB'] = (df['close'] - df['BBand_lower_band'])/(df['BBand_upper_band'] - df['BBand_lower_band'])
    df['Boll_signal'] = 0.0


    for i in range(len(df)):
        if df['close'][i] > df['BBand_lower_band'][i] and df['close'][i-1] < df['BBand_lower_band'][i-1]:#df['ROC'][i] > 0:
            df['Boll_signal'][i] = 1.0
        elif df['close'][i] < df['BBand_upper_band'][i] and df['close'][i-1] > df['BBand_upper_band'][i-1]:#df['ROC'][i] < 0
            df['Boll_signal'][i] = -1.0

    for i in range(1,len(df)): 
        if(df['Boll_signal'][i-1] != 1.0 and df['Boll_signal'][i] == 1.0):
            df['Boll_signal'][i-1] = 1.0
        elif(df['Boll_signal'][i-1] != -1.0 and df['Boll_signal'][i] == -1.0):
            df['Boll_signal'][i-1] = -1.0
        else:
            df['Boll_signal'][i-1] = 0.0

    return df['Boll_signal']

def BBAND_price(df): 
    df['Boll_signal_price']=df['Boll_signal']*df['close']
    return df['Boll_signal_price']

def BBAND_plot(df):          
    # Initialize the plot figure
    fig = plt.figure(figsize=(20,10))

    # Add a subplot and label for y-axis
    ax1 = fig.add_subplot(111,  ylabel='Price in $')

    # Plot the closing price
    df.plot(x='datetime',y=['close'],ax=ax1, lw=1.)

    # Plot the buy signals
    ax1.plot(df.loc[df.Boll_signal == 1.0].index, 
             df.close[df.Boll_signal == 1.0],
             '^', markersize=15, color='green')

    # Plot the sell signals
    ax1.plot(df.loc[df.Boll_signal == -1.0].index, 
             df.close[df.Boll_signal == -1.0],
             'v', markersize=15, color='red')

    # Show the plot
    plt.show()

In [5]:
def ATRRSI(df):
    df['RSI'] = talib.RSI(df['close'],timeperiod=1950)
    df['RSI_signal']=0
    df.drop(['RSI_signal'],axis=1,inplace=True)

    RSI = df['RSI']


    RSI_signal = np.zeros(df.shape[0])

    for i in range(len(RSI)-1):

        if(80 < RSI[i] < 100):
            RSI_signal[i] = -1.0
        elif (20 < RSI[i] < 50):
            RSI_signal[i] = -1.0
        elif(50 < RSI[i] < 80) :
            RSI_signal[i] = 1.0
        elif(0 < RSI[i] < 20):
            RSI_signal[i] = 1.0
        else:
            RSI_signal[i] = 0.0

    for i in range(1,len(RSI)): 
        if(RSI_signal[i-1] != 1.0 and RSI_signal[i] == 1.0):
            RSI_signal[i-1] = 1.0
        elif(RSI_signal[i-1] != -1.0 and RSI_signal[i] == -1.0):
            RSI_signal[i-1] = -1.0
        else:
            RSI_signal[i-1] = 0.0

    df.insert(1,'RSI_signal', RSI_signal)

    return df['RSI_signal']

def ATRRSI_price(df): 
    df['ATRRSI_signal_price']=df['RSI_signal']*df['close']
    return df['ATRRSI_signal_price']


def ATRRSI_plot(df):
    # Initialize the plot figure
    fig = plt.figure(figsize=(20,10))

    # Add a subplot and label for y-axis
    ax1 = fig.add_subplot(111,  ylabel='Price in $')

    # Plot the closing price
    df.plot(x='datetime',y=['close'],ax=ax1, lw=1.)

    # Plot the buy signals
    ax1.plot(df.loc[df.RSI_signal == 1.0].index, 
             df.close[df.RSI_signal == 1.0],
             '^', markersize=10, color='green')

    # Plot the sell signals
    ax1.plot(df.loc[df.RSI_signal == -1.0].index, 
             df.close[df.RSI_signal == -1.0],
             'v', markersize=10, color='red')

    # Show the plot
    plt.show()

In [6]:
def ATR(df):
    df['RSI'] = talib.RSI(df['close'],timeperiod=1950)
    df['ATR_daily'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=3900)
    df['ATR_MA'] = talib.MA(df['ATR_daily'], timeperiod=1950)
    df['Trading_Zone'] = 0.0
    df.drop(['Trading_Zone'],axis=1,inplace=True)
    Trading_Zone = np.zeros(df.shape[0])
    for i in range(len(df)):
        if df.ATR_daily[i] > df.ATR_MA[i]:
            Trading_Zone[i] = 1
        else:
            Trading_Zone[i] = 0

    df.insert(1,'Trading_Zone', Trading_Zone)
    df['ATR_RSI_signal'] = 0.0
    for i in range(len(df)):
        if df['Trading_Zone'][i] == 1:
            if df['RSI'][i] >= 51:
                df['ATR_RSI_signal'][i] = 1.0
            elif df['RSI'][i] <= 49:
                df['ATR_RSI_signal'][i] = -1.0


    return df['ATR_RSI_signal']
def ATR_price(df): 
    df['ATR_RSI_signal_price']=df['ATR_RSI_signal']*df['close']
    return df['ATR_RSI_signal_price']



def ATR_plot(df):
    fig = plt.figure(figsize=(20,10))

    # Add a subplot and label for y-axis
    ax1 = fig.add_subplot(111,  ylabel='Price in $')

    # Plot the closing price
    df['close'].plot(ax=ax1, color='r', lw=1.)

    # Plot the BBAND
    df.plot(x='datetime',y=['close'],ax=ax1, lw=1.)

    # Plot the buy signals
    ax1.plot(df.loc[df.ATR_RSI_signal == 1.0].index, 
             df.close[df.ATR_RSI_signal == 1.0],
             '^', markersize=10, color='green')

    # Plot the sell signals
    ax1.plot(df.loc[df.ATR_RSI_signal == -1.0].index, 
             df.close[df.ATR_RSI_signal == -1.0],
             'v', markersize=10, color='red')

    # Show the plot
    plt.show()

In [7]:
def Fibonacci(df):
    df['fibonacci_signal']=0.0
    timeperiod = 3900
    for i in range(len(df['close'])-2*timeperiod):
        start = df['close'][i]
        end = df['close'][i+timeperiod]
        nextdata = df['close'][i+2*timeperiod]
        if start >= end:

            if (nextdata-end) >=(start - end)*0.618:
                df['fibonacci_signal'][i+timeperiod]=1.0
            else:
                df['fibonacci_signal'][i+timeperiod]=0.0
        else:
            if (end-nextdata) >= (end - start)*0.618:
                df['fibonacci_signal'][i+timeperiod]=-1.0
            else:
                df['fibonacci_signal'][i+timeperiod]=0.0

    return df['fibonacci_signal']


def Fibonacci_price(df): 
    df['fibonacci_signal_price']=df['fibonacci_signal']*df['close']
    return df['fibonacci_signal_price']

def Fibonacci_plot(df):
    fig = plt.figure(figsize=(20,10))

    # Add a subplot and label for y-axis
    ax1 = fig.add_subplot(111,  ylabel='Price in $')

    # Plot the closing price
    df['close'].plot(ax=ax1, color='r', lw=1.)

    # Plot the BBAND
    df.plot(x='datetime',y=['close'],ax=ax1, lw=1.)

    # Plot the buy signals
    ax1.plot(df.loc[df.fibonacci_signal == 1.0].index, 
             df.close[df.fibonacci_signal == 1.0],
             '^', markersize=10, color='green')

    # Plot the sell signals
    ax1.plot(df.loc[df.fibonacci_signal== -1.0].index, 
             df.close[df.fibonacci_signal == -1.0],
             'v', markersize=10, color='red')

    # Show the plot
    plt.show()

In [8]:
def EMA(df):
    df['EMA5'] = talib.EMA(df['close'], timeperiod = 1950)
    df['ma5']=df['close'].rolling(window=1950,center=False).mean() 
    df['EMA_signal']=0.0
    for i in range(len(df['EMA5'])-1):
        if abs(df['EMA5'][i+1]-df['ma5'][i+1])<=0.2:
            if df['EMA5'][i]>df['EMA5'][i+1]:
                df['EMA_signal'][i+1]=1.0
            elif df['EMA5'][i]<df['EMA5'][i+1]:
                df['EMA_signal'][i+1]=-1.0

    return df['EMA_signal']
       
def EMA_price(df): 
    df['EMA_signal_price']=df['EMA_signal']*df['close']
    return df['EMA_signal_price']
                
def EMA_plot(df):
    fig = plt.figure(figsize=(20,10))

    # Add a subplot and label for y-axis
    ax1 = fig.add_subplot(111,  ylabel='Price in $')

    # Plot the closing price
    df['close'].plot(ax=ax1, color='r', lw=1.)

    # Plot the BBAND
    df.plot(x='datetime',y=['close'],ax=ax1, lw=1.)

    # Plot the buy signals
    ax1.plot(df.loc[df.EMA_signal == 1.0].index, 
             df.close[df.EMA_signal == 1.0],
             '^', markersize=10, color='green')

    # Plot the sell signals
    ax1.plot(df.loc[df.EMA_signal == -1.0].index, 
             df.close[df.EMA_signal == -1.0],
             'v', markersize=10, color='red')

    # Show the plot
    plt.show()

In [9]:
ADBE = pd.read_csv("Intraday Historical/20180331_20200601_ABDE.csv").fillna(method='ffill')
ANSS = pd.read_csv("Intraday Historical/20180331_20200601_ANSS.csv").fillna(method='ffill')
ATVI = pd.read_csv("Intraday Historical/20180331_20200601_ATVI.csv").fillna(method='ffill')
CTXS = pd.read_csv("Intraday Historical/20180331_20200601_CTXS.csv").fillna(method='ffill')
DIS = pd.read_csv("Intraday Historical/20180331_20200601_DIS.csv").fillna(method='ffill')

DOCU = pd.read_csv("Intraday Historical/20180331_20200601_DOCU.csv").fillna(method='ffill')
EA = pd.read_csv("Intraday Historical/20180331_20200601_EA.csv").fillna(method='ffill')
FB = pd.read_csv("Intraday Historical/20180331_20200601_FB.csv").fillna(method='ffill')
INTU = pd.read_csv("Intraday Historical/20180331_20200601_INTU.csv").fillna(method='ffill')
MSFT = pd.read_csv("Intraday Historical/20180331_20200601_MSFT.csv").fillna(method='ffill')

ORCL = pd.read_csv("Intraday Historical/20180331_20200601_ORCL.csv").fillna(method='ffill')
PAYC = pd.read_csv("Intraday Historical/20180331_20200601_PAYC.csv").fillna(method='ffill')
RNG = pd.read_csv("Intraday Historical/20180331_20200601_RNG.csv").fillna(method='ffill')
TEAM = pd.read_csv("Intraday Historical/20180331_20200601_TEAM.csv").fillna(method='ffill')
TTWO = pd.read_csv("Intraday Historical/20180331_20200601_TTWO.csv").fillna(method='ffill')

TWTR = pd.read_csv("Intraday Historical/20180331_20200601_TWTR.csv").fillna(method='ffill')
VEEV = pd.read_csv("Intraday Historical/20180331_20200601_VEEV.csv").fillna(method='ffill')
OKTA = pd.read_csv("Intraday Historical/20180331_20200601_OKTA.csv").fillna(method='ffill')
Portfolio = pd.read_csv("Daily Data/daily_new_Portfolio.csv").fillna(method='ffill')
minute_datetime=pd.read_csv('Intraday Historical/minute_datetime.csv')


In [10]:
# ABDE_ANSS = pd.merge(ADBE,ANSS,how='outer', on = 'datetime',sort=True)
# Merge_ATVI = pd.merge(ABDE_ANSS,ATVI,how='outer', on = 'datetime',sort=True)
# Merge_CTXS = pd.merge(Merge_ATVI,CTXS,how='outer', on = 'datetime',sort=True)
# Merge_DIS = pd.merge(Merge_CTXS,DIS,how='outer', on = 'datetime',sort=True)

# Merge_DOCU = pd.merge(Merge_DIS,DOCU,how='outer', on = 'datetime',sort=True)
# Merge_EA = pd.merge(Merge_DOCU,EA,how='outer', on = 'datetime',sort=True)
# Merge_FB = pd.merge(Merge_EA,FB,how='outer', on = 'datetime',sort=True)
# Merge_INTU = pd.merge(Merge_FB,INTU,how='outer', on = 'datetime',sort=True)
# Merge_MSFT = pd.merge(Merge_INTU,MSFT,how='outer', on = 'datetime',sort=True)

# Merge_ORCL = pd.merge(Merge_MSFT,ORCL,how='outer', on = 'datetime',sort=True)
# Merge_PAYC = pd.merge(Merge_ORCL,PAYC,how='outer', on = 'datetime',sort=True)
# Merge_RNG = pd.merge(Merge_PAYC,RNG,how='outer', on = 'datetime',sort=True)
# Merge_TEAM = pd.merge(Merge_RNG,TEAM,how='outer', on = 'datetime',sort=True)
# Merge_TTWO = pd.merge(Merge_TEAM,TTWO,how='outer', on = 'datetime',sort=True)

# Merge_TWTR = pd.merge(Merge_TTWO,TWTR,how='outer', on = 'datetime',sort=True)
# Merge_VEEV = pd.merge(Merge_TWTR,VEEV,how='outer', on = 'datetime',sort=True)
# Merge_minute_data = pd.merge(Merge_VEEV,OKTA,how='outer', on = 'datetime',sort=True)

In [11]:
# ABDE.to_csv("ABDE.csv", index=False, encoding='utf-8')
# ANSS.to_csv("ANSS.csv", index=False, encoding='utf-8')
# ATVI.to_csv("ATVI.csv", index=False, encoding='utf-8')
# CTXS.to_csv("CTXS.csv", index=False, encoding='utf-8')
# DIS.to_csv("DIS.csv", index=False, encoding='utf-8')

# DOCU.to_csv("DOCU.csv", index=False, encoding='utf-8')
# EA.to_csv("EA.csv", index=False, encoding='utf-8')
# FB.to_csv("FB.csv", index=False, encoding='utf-8')
# INTU.to_csv("INTU.csv", index=False, encoding='utf-8')
# MSFT.to_csv("MSFT.csv", index=False, encoding='utf-8')

# ORCL.to_csv("ORCL.csv", index=False, encoding='utf-8')
# PAYC.to_csv("PAYC.csv", index=False, encoding='utf-8')
# RNG.to_csv("RNG.csv", index=False, encoding='utf-8')
# TEAM.to_csv("TEAM.csv", index=False, encoding='utf-8')
# TTWO.to_csv("TTWO.csv", index=False, encoding='utf-8')

# TWTR.to_csv("TWTR.csv", index=False, encoding='utf-8')
# VEEV.to_csv("VEEV.csv", index=False, encoding='utf-8')
# OKTA.to_csv("OKTA.csv", index=False, encoding='utf-8')

In [12]:
output=pd.read_csv('Intraday Historical/minute_datetime.csv')
output_price=pd.read_csv('Intraday Historical/minute_datetime.csv')
def indicator(ticker,name):
    output[name+'_MA_signal'] = MA(ticker)
    output_price[name+'_MA_price']=MA_price(ticker)
    output[name+'_MACD_signal'] = MACD(ticker)
    output_price[name+'_MACD_price']=MACD_price(ticker)
    output[name+'_BBAND_signal'] = BBAND(ticker)
    output_price[name+'_BBAND_price']=BBAND_price(ticker)
    output[name+'_ATRRSI_signal'] = ATRRSI(ticker)
    output_price[name+'_ATRRSI_price']=ATRRSI_price(ticker)
    output[name+'_ATR_signal'] = ATR(ticker)
    output_price[name+'_ATR_price']=ATR_price(ticker)
    output[name+'_Fibonacci_signal'] = Fibonacci(ticker)
    output_price[name+'_Fibonacci_price']=Fibonacci_price(ticker)
    output[name+'_EMA_signal'] = EMA(ticker)
    output_price[name+'_EMA_price']=EMA_price(ticker)
    output[name+'_return'] = ticker['close']- ticker['close'].shift(1)
    output[name+'_price'] = ticker['close']




In [13]:
indicator(ADBE,'ADBE')
print('1/18')
indicator(ANSS,'ANSS')
indicator(ATVI,'ATVI')
print('3/18')
indicator(CTXS,'CTXS')
indicator(DIS,'DIS')
print('5/18')
indicator(DOCU,'DOCU')
indicator(EA,'EA')
print('7/18')
indicator(FB,'FB')
indicator(INTU,'INTU')
print('9/18')
indicator(MSFT,'MSFT')

indicator(ORCL,'ORCL')
print('11/18')
indicator(PAYC,'PAYC')
indicator(RNG,'RNG')
print('13/18')
indicator(TEAM,'TEAM')
indicator(TTWO,'TTWO')
print('15/18')

indicator(TWTR,'TWTR')
indicator(VEEV,'VEEV')
print('17/18')
indicator(OKTA,'OKTA')


/Users/wanrongcai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/wanrongcai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/wanrongcai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

1/18
3/18
5/18
7/18
9/18
11/18
13/18
15/18
17/18


/Users/wanrongcai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#output_price.to_csv("output.csv", index=False, encoding='utf-8')

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
def random_forest(ticker,name):
    
    df_x = output[[name+'_MA_signal',name+'_MACD_signal',name+'_BBAND_signal',name+'_ATRRSI_signal',name+'_ATR_signal',name+'_Fibonacci_signal',name+'_EMA_signal']]
    df_y = output[name+'_return']
    
    X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, random_state=42)
    
    rf = RandomForestRegressor()
    rf.fit(X_train.dropna(),y_train.dropna())
    
    ypred_rf = rf.predict(X_test)
    importances = rf.feature_importances_

    indices = np.argsort(importances)[::-1]

    feature_name = df_x.columns

    for f in indices[0:3]:
        print(feature_name[f],"Improtances:", importances[f])

    return "done"

In [15]:
print('Random Forest in ADBE',random_forest(ADBE,'ADBE'))
print('Random Forest in ANSS',random_forest(ANSS,'ANSS'))
print('Random Forest in ATVI',random_forest(ATVI,'ATVI'))
print('Random Forest in CTXS',random_forest(CTXS,'CTXS'))
print('Random Forest in DIS',random_forest(DIS,'DIS'),"5/18")

print('Random Forest in DOCU',random_forest(DOCU,'DOCU'))
print('Random Forest in EA',random_forest(EA,'EA'))
print('Random Forest in FB',random_forest(FB,'FB'))
print('Random Forest in INTU',random_forest(INTU,'INTU'))
print('Random Forest in MSFT',random_forest(MSFT,'MSFT'),"10/18")

print('Random Forest in ORCL',random_forest(ORCL,'ORCL'))
print('Random Forest in PAYC',random_forest(PAYC,'PAYC'))
print('Random Forest in RNG',random_forest(RNG,'RNG'))
print('Random Forest in TEAM',random_forest(TEAM,'TEAM'))
print('Random Forest in TTWO',random_forest(TTWO,'TTWO'),"15/18")

print('Random Forest in TWTR',random_forest(TWTR,'TWTR'))
print('Random Forest in VEEV',random_forest(VEEV,'VEEV'))
print('Random Forest in OKTA',random_forest(OKTA,'OKTA'),"finished")


ADBE_ATR_signal Improtances: 0.30798844146846716
ADBE_Fibonacci_signal Improtances: 0.216339226369432
ADBE_EMA_signal Improtances: 0.15516976195222507
Random Forest in ADBE done
ANSS_Fibonacci_signal Improtances: 0.272795426492263
ANSS_EMA_signal Improtances: 0.21090938263304962
ANSS_ATR_signal Improtances: 0.16895102622089123
Random Forest in ANSS done
ATVI_Fibonacci_signal Improtances: 0.24043307302785621
ATVI_ATR_signal Improtances: 0.21063302290664163
ATVI_EMA_signal Improtances: 0.1773851426651042
Random Forest in ATVI done
CTXS_Fibonacci_signal Improtances: 0.3257329804961108
CTXS_EMA_signal Improtances: 0.21189620510196217
CTXS_ATR_signal Improtances: 0.1304977354552678
Random Forest in CTXS done
DIS_ATR_signal Improtances: 0.35384960860399794
DIS_EMA_signal Improtances: 0.2579582791910009
DIS_Fibonacci_signal Improtances: 0.17625801222316767
Random Forest in DIS done 5/18
DOCU_EMA_signal Improtances: 0.3036335977618047
DOCU_ATR_signal Improtances: 0.26000616562250295
DOCU_Fibon

In [18]:
# minute_rate=pow(3100.29/2640.87,1/224245)
# output[1000][1]=pow(1+minute_rate,i)
# output['interest_rate']=1

# for i in range(len(output['interest_rate'])-1):
#     output.iloc['interest_rate'][i] = output.iloc['interest_rate'][i-1]*minute_rate:



# for i in range(224245):
#     output[1000][1]=pow(1+minute_rate,i)
def dollar_weighted_return(ticker,name): #parameter, string
    pn=sum(output_price[name+'_MA_price'].fillna(0))
    pp=sum(abs(output_price[name+'_MA_price'].fillna(0)))
    output[name+'_MA_Investment']=(pp+pn)/2
    output[name+'_MA_DWR']=pn
    
    pn=sum(output_price[name+'_MACD_price'].fillna(0))
    pp=sum(abs(output_price[name+'_MACD_price'].fillna(0)))
    output[name+'_MACD_Investment']=(pp+pn)/2
    output[name+'_MACD_DWR']=pn
    
    pn=sum(output_price[name+'_BBAND_price'].fillna(0))
    pp=sum(abs(output_price[name+'_BBAND_price'].fillna(0)))
    output[name+'_BBAND_Investment']=(pp+pn)/2
    output[name+'_BBAND_DWR']=pn
    
    pn=sum(output_price[name+'_ATRRSI_price'].fillna(0))
    pp=sum(abs(output_price[name+'_ATRRSI_price'].fillna(0)))
    output[name+'_ATRRSI_Investment']=(pp+pn)/2
    output[name+'_ATRRSI_DWR']=pn
    
    pn=sum(output_price[name+'_ATR_price'].fillna(0))
    pp=sum(abs(output_price[name+'_ATR_price'].fillna(0)))
    output[name+'_ATR_Investment']=(pp+pn)/2
    output[name+'_ATR_DWR']=pn
    
    pn=sum(output_price[name+'_Fibonacci_price'].fillna(0))
    pp=sum(abs(output_price[name+'_Fibonacci_price'].fillna(0)))
    output[name+'_Fibonacci_Investment']=(pp+pn)/2
    output[name+'_Fibonacci_DWR']=pn
    
    pn=sum(output_price[name+'_EMA_price'].fillna(0))
    pp=sum(abs(output_price[name+'_EMA_price'].fillna(0)))
    output[name+'_EMA_Investment']=(pp+pn)/2
    output[name+'_EMA_DWR']=pn
    


    

In [19]:
dollar_weighted_return(ADBE,'ADBE')

dollar_weighted_return(ANSS,'ANSS')
dollar_weighted_return(ATVI,'ATVI')

dollar_weighted_return(CTXS,'CTXS')
dollar_weighted_return(DIS,'DIS')

dollar_weighted_return(DOCU,'DOCU')
dollar_weighted_return(EA,'EA')

dollar_weighted_return(FB,'FB')
dollar_weighted_return(INTU,'INTU')

dollar_weighted_return(MSFT,'MSFT')

dollar_weighted_return(ORCL,'ORCL')

dollar_weighted_return(PAYC,'PAYC')
dollar_weighted_return(RNG,'RNG')

dollar_weighted_return(TEAM,'TEAM')
dollar_weighted_return(TTWO,'TTWO')


dollar_weighted_return(TWTR,'TWTR')
dollar_weighted_return(VEEV,'VEEV')

dollar_weighted_return(OKTA,'OKTA')

In [20]:
output.to_csv("output.csv", index=False, encoding='utf-8')

In [21]:
# minute_rate=pow(3100.29/2640.87,1/224245)
# output['interest_rate']=1

# for i in range(len(output['interest_rate'])-1):
#     output.iloc['interest_rate'][i] = output.iloc['interest_rate'][i-1]*minute_rate